Objective:
 
We initially believed there is likely to be an association with deprivation levels and rates of injury. We will do this by comparing 1 month of NHS maternity dataset - trauma reported to the deprivation deciles at birth and observe if there is a positive correlation from there.

After doing exploratory data analysis we found there was a weak negative correlation between deprivation levels and injury rates, finding richer areas were more likely to incur injuries while giving birth.

Scope creep - From this the objective has changed, we will thoroughly analyse almost every element of the NHS maternity dataset to see if we can find what factors influence injury rates the most.


Accounting for misreporting:

From previous analysis we are aware there is misreporting of trauma data, with some trusts reporting that they have no trauma in their maternity wards.
There are other missing values and misreportings among the other elements of the dataset but it is not possible to filter this out unless analysing the data on a more granular scale. This scope creep will be avoided as otherwise the analysis will never finish and will likely result in the data being so incomplete nothing can be drawn from it.

Due to the offical nature of the reporting it is highly unlikely trusts will over-report trauma.

Update - we will only remove trusts that have trauma rates under 10% as there are only 3 clear cases of misreporting.

These trusts are:
IMPERIAL COLLEGE HEALTHCARE NHS TRUST,
ROYAL FREE LONDON NHS FOUNDATION TRUST,
THE SHREWSBURY AND TELFORD HOSPITAL NHS TRUST


Results:

We have found that the Pearson correlation coefficient is 0.14, with the p-value associated with this being 0.02. As the p-value is lower than 0.05 we reject the mull hypothesis, finding a weak correlation between deprivation and trauma rates

mean_x: 5.058180233511137, mean_y: 64.31040402493487
covariance: 236.2767763568413
std_x: 13.676746245764898, std_y: 119.08114980426666
Pearson correlation coefficient: 0.14507588492112683
P-value: 0.021158937545157963




In [115]:
import pandas as pd
import glob
import numpy as np
import os
from datetime import datetime
import bokeh
import pandas_bokeh
from bokeh.plotting import figure, output_notebook, show
from bokeh.layouts import column, row
from bokeh.models import ColumnDataSource, HoverTool, LabelSet
from bokeh.transform import cumsum
from bokeh.palettes import Category20c
from math import pi
from IPython.display import FileLink
from scipy.stats import t

In [50]:
#Initalises our maternity dataset
pd.set_option('plotting.backend', 'pandas_bokeh')
output_notebook()
file_path = 'C:/NHS maternity data/2023/msds-apr2023-exp-data-final.csv'

df = pd.read_csv(file_path)
df.head()

Loading BokehJS ...

,ReportingPeriodStartDate,ReportingPeriodEndDate,Dimension,Org_Level,Org_Code,Org_Name,Measure,Count_Of,Final_value
0,01/04/2023,30/04/2023,AgeAtBookingMotherAvg,National,ALL,ALL SUBMITTERS,NaN,Average over women,31
1,01/04/2023,30/04/2023,AgeAtBookingMotherGroup,National,ALL,ALL SUBMITTERS,20 to 24,Women,6525
2,01/04/2023,30/04/2023,AgeAtBookingMotherGroup,National,ALL,ALL SUBMITTERS,25 to 29,Women,13710
3,01/04/2023,30/04/2023,AgeAtBookingMotherGroup,National,ALL,ALL SUBMITTERS,30 to 34,Women,17640
4,01/04/2023,30/04/2023,AgeAtBookingMotherGroup,National,ALL,ALL SUBMITTERS,35 to 39,Women,9895


In [116]:
def pearson_correlation_debug(x, y):
    n = len(x)
    mean_x = sum(x) / n
    mean_y = sum(y) / n
    covariance = sum((x.iloc[i] - mean_x) * (y.iloc[i] - mean_y) for i in range(n))
    std_x = (sum((x.iloc[i] - mean_x) ** 2 for i in range(n)) ** 0.5)
    std_y = (sum((y.iloc[i] - mean_y) ** 2 for i in range(n)) ** 0.5)
    if std_x == 0 or std_y == 0:
        return float('nan')
    r = covariance / (std_x * std_y)
    return r


In [120]:
def pearson_correlation_with_pvalue_debug(x, y):
    n = len(x)
    r = pearson_correlation_debug(x, y)
    if pd.isna(r):  # Check if r is NaN
        print("Pearson correlation coefficient is NaN.")
        return r, float('nan')
    
    # Handle the case where r is exactly 1 or -1
    if r == 1 or r == -1:
        print("Pearson correlation coefficient is exactly 1 or -1.")
        return r, 0.0  # P-value is 0 because the correlation is perfect
    
    # t-statistic
    t_stat = r * ((n - 2) / (1 - r ** 2)) ** 0.5
    
    # Calculate p-value using the accurate t-distribution function
    p_value = 2 * t.sf(abs(t_stat), df=n - 2)
    
    # Handle very small p-values
    if p_value < 1e-10:
        p_value = "< 1e-10"  # Replace with a threshold value instead of 0
    
    return r, p_value

In [55]:
#Filters dftrusts to only show NHS trusts, removing counties ect.
dftrusts = df[df['Org_Name'].str.contains('trust', case=False, na=False)]

In [56]:
# Define the measures we want to remove
unwanted_measures = [
    "Missing Value / Value outside reporting parameters",
    "Pseudo postcode recorded (includes no fixed abode or resident overseas)",
    "Resident Elsewhere in UK, Channel Islands or Isle of Man"
]
dftrusts.head(100)

,ReportingPeriodStartDate,ReportingPeriodEndDate,Dimension,Org_Level,Org_Code,Org_Name,Measure,Count_Of,Final_value
33452,01/04/2023,30/04/2023,AgeAtBookingMotherAvg,Provider,R0A,MANCHESTER UNIVERSITY NHS FOUNDATION TRUST,NaN,Average over women,31
33453,01/04/2023,30/04/2023,AgeAtBookingMotherGroup,Provider,R0A,MANCHESTER UNIVERSITY NHS FOUNDATION TRUST,20 to 24,Women,180
33454,01/04/2023,30/04/2023,AgeAtBookingMotherGroup,Provider,R0A,MANCHESTER UNIVERSITY NHS FOUNDATION TRUST,25 to 29,Women,400
33455,01/04/2023,30/04/2023,AgeAtBookingMotherGroup,Provider,R0A,MANCHESTER UNIVERSITY NHS FOUNDATION TRUST,30 to 34,Women,525
33456,01/04/2023,30/04/2023,AgeAtBookingMotherGroup,Provider,R0A,MANCHESTER UNIVERSITY NHS FOUNDATION TRUST,35 to 39,Women,285
...,...,...,...,...,...,...,...,...,...
33547,01/04/2023,30/04/2023,SmokingStatusGroupBooking,Provider,R0A,MANCHESTER UNIVERSITY NHS FOUNDATION TRUST,Non-Smoker / Ex-Smoker,Women,1095
33548,01/04/2023,30/04/2023,SmokingStatusGroupBooking,Provider,R0A,MANCHESTER UNIVERSITY NHS FOUNDATION TRUST,Smoker,Women,95
33549,01/04/2023,30/04/2023,TotalBabies,Provider,R0A,MANCHESTER UNIVERSITY NHS FOUNDATION TRUST,NaN,Babies,1210
33550,01/04/2023,30/04/2023,TotalBookings,Provider,R0A,MANCHESTER UNIVERSITY NHS FOUNDATION TRUST,NaN,Women,1525


In [57]:
dftrusts = dftrusts[~dftrusts['Measure'].isin(unwanted_measures)]
dftrusts.head(100)

,ReportingPeriodStartDate,ReportingPeriodEndDate,Dimension,Org_Level,Org_Code,Org_Name,Measure,Count_Of,Final_value
33452,01/04/2023,30/04/2023,AgeAtBookingMotherAvg,Provider,R0A,MANCHESTER UNIVERSITY NHS FOUNDATION TRUST,NaN,Average over women,31
33453,01/04/2023,30/04/2023,AgeAtBookingMotherGroup,Provider,R0A,MANCHESTER UNIVERSITY NHS FOUNDATION TRUST,20 to 24,Women,180
33454,01/04/2023,30/04/2023,AgeAtBookingMotherGroup,Provider,R0A,MANCHESTER UNIVERSITY NHS FOUNDATION TRUST,25 to 29,Women,400
33455,01/04/2023,30/04/2023,AgeAtBookingMotherGroup,Provider,R0A,MANCHESTER UNIVERSITY NHS FOUNDATION TRUST,30 to 34,Women,525
33456,01/04/2023,30/04/2023,AgeAtBookingMotherGroup,Provider,R0A,MANCHESTER UNIVERSITY NHS FOUNDATION TRUST,35 to 39,Women,285
...,...,...,...,...,...,...,...,...,...
33562,01/04/2023,30/04/2023,ApgarScore5TermGroup7,Provider,R0B,SOUTH TYNESIDE AND SUNDERLAND NHS FOUNDATION T...,7 to 10,Babies,240
33564,01/04/2023,30/04/2023,BabyFirstFeedBreastMilkStatus,Provider,R0B,SOUTH TYNESIDE AND SUNDERLAND NHS FOUNDATION T...,Maternal or Donor Breast Milk,Babies,135
33565,01/04/2023,30/04/2023,BabyFirstFeedBreastMilkStatus,Provider,R0B,SOUTH TYNESIDE AND SUNDERLAND NHS FOUNDATION T...,Not Breast Milk,Babies,135
33566,01/04/2023,30/04/2023,BirthweightTermGroup,Provider,R0B,SOUTH TYNESIDE AND SUNDERLAND NHS FOUNDATION T...,2000g to 2499g,Babies,10


In [58]:
dftrusts.loc[dftrusts['Dimension'] == 'DeprivationDecileAtBooking', 'Measure'] = (
    dftrusts.loc[dftrusts['Dimension'] == 'DeprivationDecileAtBooking', 'Measure']
    .apply(lambda x: x[:2] if len(x) > 2 else x) 
)
#Removes 'most/least' deprived segments from measure
dftrusts.loc[(dftrusts['Dimension'] == 'DeprivationDecileAtBooking') & (dftrusts['Measure'] == '01'), 'Measure'] = '1'

# Display the modified DataFrame to verify changes
dftrusts.head(100)


,ReportingPeriodStartDate,ReportingPeriodEndDate,Dimension,Org_Level,Org_Code,Org_Name,Measure,Count_Of,Final_value
33452,01/04/2023,30/04/2023,AgeAtBookingMotherAvg,Provider,R0A,MANCHESTER UNIVERSITY NHS FOUNDATION TRUST,NaN,Average over women,31
33453,01/04/2023,30/04/2023,AgeAtBookingMotherGroup,Provider,R0A,MANCHESTER UNIVERSITY NHS FOUNDATION TRUST,20 to 24,Women,180
33454,01/04/2023,30/04/2023,AgeAtBookingMotherGroup,Provider,R0A,MANCHESTER UNIVERSITY NHS FOUNDATION TRUST,25 to 29,Women,400
33455,01/04/2023,30/04/2023,AgeAtBookingMotherGroup,Provider,R0A,MANCHESTER UNIVERSITY NHS FOUNDATION TRUST,30 to 34,Women,525
33456,01/04/2023,30/04/2023,AgeAtBookingMotherGroup,Provider,R0A,MANCHESTER UNIVERSITY NHS FOUNDATION TRUST,35 to 39,Women,285
...,...,...,...,...,...,...,...,...,...
33562,01/04/2023,30/04/2023,ApgarScore5TermGroup7,Provider,R0B,SOUTH TYNESIDE AND SUNDERLAND NHS FOUNDATION T...,7 to 10,Babies,240
33564,01/04/2023,30/04/2023,BabyFirstFeedBreastMilkStatus,Provider,R0B,SOUTH TYNESIDE AND SUNDERLAND NHS FOUNDATION T...,Maternal or Donor Breast Milk,Babies,135
33565,01/04/2023,30/04/2023,BabyFirstFeedBreastMilkStatus,Provider,R0B,SOUTH TYNESIDE AND SUNDERLAND NHS FOUNDATION T...,Not Breast Milk,Babies,135
33566,01/04/2023,30/04/2023,BirthweightTermGroup,Provider,R0B,SOUTH TYNESIDE AND SUNDERLAND NHS FOUNDATION T...,2000g to 2499g,Babies,10


In [59]:
dftrusts.loc[dftrusts['Dimension'] == 'DeprivationDecileAtBooking', 'Measure'] = (
    dftrusts.loc[dftrusts['Dimension'] == 'DeprivationDecileAtBooking', 'Measure']
    .astype(int)  # Convert to integer
)
dftrusts.head(400)

,ReportingPeriodStartDate,ReportingPeriodEndDate,Dimension,Org_Level,Org_Code,Org_Name,Measure,Count_Of,Final_value
33452,01/04/2023,30/04/2023,AgeAtBookingMotherAvg,Provider,R0A,MANCHESTER UNIVERSITY NHS FOUNDATION TRUST,NaN,Average over women,31
33453,01/04/2023,30/04/2023,AgeAtBookingMotherGroup,Provider,R0A,MANCHESTER UNIVERSITY NHS FOUNDATION TRUST,20 to 24,Women,180
33454,01/04/2023,30/04/2023,AgeAtBookingMotherGroup,Provider,R0A,MANCHESTER UNIVERSITY NHS FOUNDATION TRUST,25 to 29,Women,400
33455,01/04/2023,30/04/2023,AgeAtBookingMotherGroup,Provider,R0A,MANCHESTER UNIVERSITY NHS FOUNDATION TRUST,30 to 34,Women,525
33456,01/04/2023,30/04/2023,AgeAtBookingMotherGroup,Provider,R0A,MANCHESTER UNIVERSITY NHS FOUNDATION TRUST,35 to 39,Women,285
...,...,...,...,...,...,...,...,...,...
33893,01/04/2023,30/04/2023,DeprivationDecileAtBooking,Provider,R1H,BARTS HEALTH NHS TRUST,2,Women,370
33894,01/04/2023,30/04/2023,DeprivationDecileAtBooking,Provider,R1H,BARTS HEALTH NHS TRUST,3,Women,370
33895,01/04/2023,30/04/2023,DeprivationDecileAtBooking,Provider,R1H,BARTS HEALTH NHS TRUST,4,Women,185
33896,01/04/2023,30/04/2023,DeprivationDecileAtBooking,Provider,R1H,BARTS HEALTH NHS TRUST,5,Women,75


In [60]:
# Filter dftrusts to show only rows where 'Dimension' is 'DeprivationDecileAtBooking'
dfdeprivation = dftrusts[dftrusts['Dimension'] == 'DeprivationDecileAtBooking']

# Display the filtered DataFrame
dfdeprivation.head(100)


,ReportingPeriodStartDate,ReportingPeriodEndDate,Dimension,Org_Level,Org_Code,Org_Name,Measure,Count_Of,Final_value
33485,01/04/2023,30/04/2023,DeprivationDecileAtBooking,Provider,R0A,MANCHESTER UNIVERSITY NHS FOUNDATION TRUST,2,Women,260
33486,01/04/2023,30/04/2023,DeprivationDecileAtBooking,Provider,R0A,MANCHESTER UNIVERSITY NHS FOUNDATION TRUST,3,Women,180
33487,01/04/2023,30/04/2023,DeprivationDecileAtBooking,Provider,R0A,MANCHESTER UNIVERSITY NHS FOUNDATION TRUST,4,Women,150
33488,01/04/2023,30/04/2023,DeprivationDecileAtBooking,Provider,R0A,MANCHESTER UNIVERSITY NHS FOUNDATION TRUST,5,Women,110
33489,01/04/2023,30/04/2023,DeprivationDecileAtBooking,Provider,R0A,MANCHESTER UNIVERSITY NHS FOUNDATION TRUST,6,Women,90
...,...,...,...,...,...,...,...,...,...
34428,01/04/2023,30/04/2023,DeprivationDecileAtBooking,Provider,RAE,BRADFORD TEACHING HOSPITALS NHS FOUNDATION TRUST,1,Women,260
34429,01/04/2023,30/04/2023,DeprivationDecileAtBooking,Provider,RAE,BRADFORD TEACHING HOSPITALS NHS FOUNDATION TRUST,10,Women,5
34532,01/04/2023,30/04/2023,DeprivationDecileAtBooking,Provider,RAJ,MID AND SOUTH ESSEX NHS FOUNDATION TRUST,2,Women,80
34533,01/04/2023,30/04/2023,DeprivationDecileAtBooking,Provider,RAJ,MID AND SOUTH ESSEX NHS FOUNDATION TRUST,3,Women,120


In [61]:
# Sort dfdeprivation by 'Org_Code' and then by 'Measure' in ascending order
dfdeprivation = dfdeprivation.sort_values(by=['Org_Code', 'Measure'])

# Display the sorted DataFrame to verify the sorting
dfdeprivation.head(100)


,ReportingPeriodStartDate,ReportingPeriodEndDate,Dimension,Org_Level,Org_Code,Org_Name,Measure,Count_Of,Final_value
33493,01/04/2023,30/04/2023,DeprivationDecileAtBooking,Provider,R0A,MANCHESTER UNIVERSITY NHS FOUNDATION TRUST,1,Women,445
33485,01/04/2023,30/04/2023,DeprivationDecileAtBooking,Provider,R0A,MANCHESTER UNIVERSITY NHS FOUNDATION TRUST,2,Women,260
33486,01/04/2023,30/04/2023,DeprivationDecileAtBooking,Provider,R0A,MANCHESTER UNIVERSITY NHS FOUNDATION TRUST,3,Women,180
33487,01/04/2023,30/04/2023,DeprivationDecileAtBooking,Provider,R0A,MANCHESTER UNIVERSITY NHS FOUNDATION TRUST,4,Women,150
33488,01/04/2023,30/04/2023,DeprivationDecileAtBooking,Provider,R0A,MANCHESTER UNIVERSITY NHS FOUNDATION TRUST,5,Women,110
...,...,...,...,...,...,...,...,...,...
34427,01/04/2023,30/04/2023,DeprivationDecileAtBooking,Provider,RAE,BRADFORD TEACHING HOSPITALS NHS FOUNDATION TRUST,9,Women,5
34429,01/04/2023,30/04/2023,DeprivationDecileAtBooking,Provider,RAE,BRADFORD TEACHING HOSPITALS NHS FOUNDATION TRUST,10,Women,5
34540,01/04/2023,30/04/2023,DeprivationDecileAtBooking,Provider,RAJ,MID AND SOUTH ESSEX NHS FOUNDATION TRUST,1,Women,65
34532,01/04/2023,30/04/2023,DeprivationDecileAtBooking,Provider,RAJ,MID AND SOUTH ESSEX NHS FOUNDATION TRUST,2,Women,80


To calculate our stats, we will take each measure * final value, then divide this by the total final value. This will give us our 'average deprivation' for each trust

In [62]:
# Calculate total deprivation for each Org_Name
dfdeprivation['Total Deprivation'] = dfdeprivation['Measure'] * dfdeprivation['Final_value']
total_deprivation_by_org = dfdeprivation.groupby('Org_Name')['Total Deprivation'].sum()

# Calculate total value for each Org_Name
total_value_by_org = dfdeprivation.groupby('Org_Name')['Final_value'].sum()

# Display intermediate calculations
total_deprivation_by_org


Org_Name
AIREDALE NHS FOUNDATION TRUST                                      950
AIREDALE NHS TRUST                                                 730
ASHFORD AND ST PETER'S HOSPITALS NHS FOUNDATION TRUST             1985
BARKING, HAVERING AND REDBRIDGE UNIVERSITY HOSPITALS NHS TRUST    2790
BARNSLEY HOSPITAL NHS FOUNDATION TRUST                             875
                                                                  ... 
WIRRAL UNIVERSITY TEACHING HOSPITAL NHS FOUNDATION TRUST          1000
WORCESTERSHIRE ACUTE HOSPITALS NHS TRUST                          2585
WRIGHTINGTON, WIGAN AND LEIGH NHS FOUNDATION TRUST                 790
WYE VALLEY NHS TRUST                                               830
YORK AND SCARBOROUGH TEACHING HOSPITALS NHS FOUNDATION TRUST       225
Name: Total Deprivation, Length: 124, dtype: object

In [63]:
actual_deprivation = total_deprivation_by_org / total_value_by_org
actual_deprivation_df = actual_deprivation.reset_index()
actual_deprivation_df.columns = ['Org_Name', 'Deprivation']

# Display the actual deprivation
actual_deprivation_df.head(100)

,Org_Name,Deprivation
0,AIREDALE NHS FOUNDATION TRUST,5.277778
1,AIREDALE NHS TRUST,5.214286
2,ASHFORD AND ST PETER'S HOSPITALS NHS FOUNDATIO...,7.218182
3,"BARKING, HAVERING AND REDBRIDGE UNIVERSITY HOS...",4.292308
4,BARNSLEY HOSPITAL NHS FOUNDATION TRUST,3.723404
...,...,...
95,THE PRINCESS ALEXANDRA HOSPITAL NHS TRUST,6.424242
96,"THE QUEEN ELIZABETH HOSPITAL, KING'S LYNN, NHS...",4.103448
97,THE ROTHERHAM NHS FOUNDATION TRUST,3.744186
98,THE ROYAL WOLVERHAMPTON NHS TRUST,3.613208


We now have our deprivation stats. Now we will Calculate our rates of trauma as a percentage. We will then analyse for an association between the trauma rates and our deprivation rates. Based on this we may analyse for associations between injury rates and other factors.

In [64]:
dftrauma = dftrusts[dftrusts['Dimension'] == 'GenitalTractTraumaticLesionGroup']

# Select only the required columns
dftrauma = dftrauma[['Dimension', 'Org_Name', 'Measure', 'Final_value']]

# Since you want each unique entry under Org_Name, you might consider dropping duplicates if needed
# dftrauma = dftrauma.drop_duplicates(subset=['Org_Name'])

# Display the new DataFrame
dftrauma.head(100)

,Dimension,Org_Name,Measure,Final_value
33511,GenitalTractTraumaticLesionGroup,MANCHESTER UNIVERSITY NHS FOUNDATION TRUST,At least one traumatic lesion,255
33512,GenitalTractTraumaticLesionGroup,MANCHESTER UNIVERSITY NHS FOUNDATION TRUST,No traumatic lesion reported,390
33617,GenitalTractTraumaticLesionGroup,SOUTH TYNESIDE AND SUNDERLAND NHS FOUNDATION T...,At least one traumatic lesion,170
33618,GenitalTractTraumaticLesionGroup,SOUTH TYNESIDE AND SUNDERLAND NHS FOUNDATION T...,No traumatic lesion reported,30
33724,GenitalTractTraumaticLesionGroup,UNIVERSITY HOSPITALS DORSET NHS FOUNDATION TRUST,At least one traumatic lesion,110
...,...,...,...,...
38471,GenitalTractTraumaticLesionGroup,ST GEORGE'S UNIVERSITY HOSPITALS NHS FOUNDATIO...,At least one traumatic lesion,110
38472,GenitalTractTraumaticLesionGroup,ST GEORGE'S UNIVERSITY HOSPITALS NHS FOUNDATIO...,No traumatic lesion reported,90
38582,GenitalTractTraumaticLesionGroup,SOUTH WARWICKSHIRE UNIVERSITY NHS FOUNDATION T...,At least one traumatic lesion,115
38583,GenitalTractTraumaticLesionGroup,SOUTH WARWICKSHIRE UNIVERSITY NHS FOUNDATION T...,No traumatic lesion reported,30


In [65]:

# Group by 'Org_Name' and 'Measure', and sum up 'Final_value'
dftrauma2 = dftrauma.groupby(['Org_Name', 'Measure'])['Final_value'].sum().unstack()

# Calculate total Final_value for each Org_Name
dftrauma2['Total'] = dftrauma2.sum(axis=1)

# Display grouped to verify

# Calculate the trauma percentage
dftrauma2['Trauma Percentage'] = (dftrauma2['At least one traumatic lesion'] / dftrauma2['Total']) * 100
columns = dftrauma2.columns.tolist()



# Assign the modified list back to the DataFrame's columns
dftrauma2.columns = columns

# Display to verify
dftrauma2.head(200)


,At least one traumatic lesion,No traumatic lesion reported,Total,Trauma Percentage
Org_Name,,,,
AIREDALE NHS FOUNDATION TRUST,20.0,60.0,80.0,25.000000
AIREDALE NHS TRUST,20.0,55.0,75.0,26.666667
ASHFORD AND ST PETER'S HOSPITALS NHS FOUNDATION TRUST,80.0,35.0,115.0,69.565217
"BARKING, HAVERING AND REDBRIDGE UNIVERSITY HOSPITALS NHS TRUST",170.0,140.0,310.0,54.838710
BARNSLEY HOSPITAL NHS FOUNDATION TRUST,85.0,50.0,135.0,62.962963
...,...,...,...,...
WIRRAL UNIVERSITY TEACHING HOSPITAL NHS FOUNDATION TRUST,75.0,60.0,135.0,55.555556
WORCESTERSHIRE ACUTE HOSPITALS NHS TRUST,155.0,70.0,225.0,68.888889
"WRIGHTINGTON, WIGAN AND LEIGH NHS FOUNDATION TRUST",30.0,25.0,55.0,54.545455


Here we find an example of misreporting. If you look in the dataframe above and sort by trauma percentage ascending you'll find Imperial College Healthcare, Royal Free London NHS foundation trust and Shrewsbury and Telford Hospital report trauma at ridiculously low rates. This data is certainly incorrect. For the others it may be possible that they have such low injury rates as they have such low numbers in their maternity suite.

In [66]:
print("Index:", dftrauma2.index.names)
print("Columns:", dftrauma2.columns)

Index: ['Org_Name']
Columns: Index(['At least one traumatic lesion', 'No traumatic lesion reported',
       'Total', 'Trauma Percentage'],
      dtype='object')


In [67]:
# Reset the index to make 'Org_Name' a column
dftrauma2.reset_index(inplace=True)

# Verify that 'Org_Name' is now a column
print(dftrauma2.columns)

final_df = actual_deprivation_df.merge(dftrauma2[['Org_Name', 'Trauma Percentage']], on='Org_Name', how='left')

# Display the final DataFrame to verify
print(final_df.head())

Index(['Org_Name', 'At least one traumatic lesion',
       'No traumatic lesion reported', 'Total', 'Trauma Percentage'],
      dtype='object')
                                            Org_Name Deprivation  \
0                      AIREDALE NHS FOUNDATION TRUST    5.277778   
1                                 AIREDALE NHS TRUST    5.214286   
2  ASHFORD AND ST PETER'S HOSPITALS NHS FOUNDATIO...    7.218182   
3  BARKING, HAVERING AND REDBRIDGE UNIVERSITY HOS...    4.292308   
4             BARNSLEY HOSPITAL NHS FOUNDATION TRUST    3.723404   

   Trauma Percentage  
0          25.000000  
1          26.666667  
2          69.565217  
3          54.838710  
4          62.962963  


In [68]:
final_df.head(100)

,Org_Name,Deprivation,Trauma Percentage
0,AIREDALE NHS FOUNDATION TRUST,5.277778,25.000000
1,AIREDALE NHS TRUST,5.214286,26.666667
2,ASHFORD AND ST PETER'S HOSPITALS NHS FOUNDATIO...,7.218182,69.565217
3,"BARKING, HAVERING AND REDBRIDGE UNIVERSITY HOS...",4.292308,54.838710
4,BARNSLEY HOSPITAL NHS FOUNDATION TRUST,3.723404,62.962963
...,...,...,...
95,THE PRINCESS ALEXANDRA HOSPITAL NHS TRUST,6.424242,61.538462
96,"THE QUEEN ELIZABETH HOSPITAL, KING'S LYNN, NHS...",4.103448,75.000000
97,THE ROTHERHAM NHS FOUNDATION TRUST,3.744186,75.000000
98,THE ROYAL WOLVERHAMPTON NHS TRUST,3.613208,69.767442


In [69]:
final_df = final_df[final_df['Trauma Percentage'] >= 27]

The code above drops the data for:
IMPERIAL COLLEGE HEALTHCARE NHS TRUST           2.3%
ROYAL FREE LONDON NHS FOUNDATION TRUST          3.07%   
THE SHREWSBURY AND TELFORD HOSPITAL NHS TRUST   8.52%
SHEFFIELD TEACHING HOSPITALS NHS FOUNDATION TRUST 24% (Very round number?)
AIREDALE NHS FOUNDATION TRUST   25%
AIREDALE NHS TRUST  26.66%
This is due to the exceedingly low rates of reported trauma


In [70]:
#This ruins the correlation code for some reason
# Calculate deciles for trauma percentage
#final_df['Decile'] = pd.qcut(final_df['Trauma Percentage'], 10, labels=False) + 1  # +1 to make deciles start from 1



#final_df['Deprivation'] = pd.to_numeric(final_df['Deprivation'], errors='coerce')
#final_df['Trauma Percentage'] = pd.to_numeric(final_df['Trauma Percentage'], errors='coerce')
#print(final_df['Deprivation'].dtype)
#print(final_df['Trauma Percentage'].dtype)
#final_df.head(100)


In [74]:
final_df = final_df.dropna(subset=['Trauma Percentage'])
x = final_df['Deprivation']
y = final_df['Trauma Percentage']

r, p_value = pearson_correlation_with_pvalue_debug(x, y)
print("Pearson correlation coefficient:", r)
print("P-value:", p_value)

mean_x: 5.058180233511137, mean_y: 64.31040402493487
covariance: 236.2767763568413
std_x: 13.676746245764898, std_y: 119.08114980426666
Pearson correlation coefficient: 0.14507588492112683
P-value: 0.021158937545157963


In [75]:
def calculate_percentage(part, total):
    if total == 0:
        return None
    percentage = (part / total) * 100
    return percentage

As a higher number in deprivation means an area is less deprived, this is interesting. Lets invert our deprivation numbers.

In [77]:
final_df['Inverted Deprivation'] = final_df['Deprivation'].max() - final_df['Deprivation']

# Calculate the Pearson correlation with the inverted values
x = final_df['Inverted Deprivation']
y = final_df['Trauma Percentage']

r, p_value = pearson_correlation_with_pvalue_debug(x, y)
print("Pearson correlation coefficient (with inverted deprivation):", r)
print("P-value:", p_value)


mean_x: 2.9910000943577155, mean_y: 64.31040402493487
covariance: -236.2767763568412
std_x: 13.6767462457649, std_y: 119.08114980426666
Pearson correlation coefficient (with inverted deprivation): -0.14507588492112675
P-value: 0.021158937545157963


Unfortunately as there is clearly an association now we must find out why. We hypothesise that birth weights and age of mother at birth could have a stronger correlation to trauma, and that in richer areas mothers give birth later (potentially to larger babies?).

In [78]:
df_age_at_booking = dftrusts[dftrusts['Dimension'] == 'AgeAtBookingMotherAvg']
df_age_at_booking = df_age_at_booking[['Org_Name', 'Dimension', 'Final_value']]
print(df_age_at_booking.head())


                                                Org_Name  \
33452         MANCHESTER UNIVERSITY NHS FOUNDATION TRUST   
33552  SOUTH TYNESIDE AND SUNDERLAND NHS FOUNDATION T...   
33659   UNIVERSITY HOSPITALS DORSET NHS FOUNDATION TRUST   
33768                            ISLE OF WIGHT NHS TRUST   
33860                             BARTS HEALTH NHS TRUST   

                   Dimension  Final_value  
33452  AgeAtBookingMotherAvg           31  
33552  AgeAtBookingMotherAvg           29  
33659  AgeAtBookingMotherAvg           31  
33768  AgeAtBookingMotherAvg           29  
33860  AgeAtBookingMotherAvg           30  


In [79]:
final_df_with_age = final_df.merge(df_age_at_booking[['Org_Name', 'Final_value']], on='Org_Name', how='left')

# Step 3: Rename the 'Final_value' column to 'Avg age'
final_df_with_age.rename(columns={'Final_value': 'Avg age'}, inplace=True)

# Display the updated DataFrame to verify
final_df_with_age.head()

,Org_Name,Deprivation,Trauma Percentage,Inverted Deprivation,Avg age
0,ASHFORD AND ST PETER'S HOSPITALS NHS FOUNDATIO...,7.218182,69.565217,0.830999,31
1,"BARKING, HAVERING AND REDBRIDGE UNIVERSITY HOS...",4.292308,54.838710,3.756873,30
2,BARNSLEY HOSPITAL NHS FOUNDATION TRUST,3.723404,62.962963,4.325776,29
3,BARTS HEALTH NHS TRUST,3.22619,36.434109,4.82299,30
4,BEDFORDSHIRE HOSPITALS NHS FOUNDATION TRUST,5.4,77.586207,2.64918,31


In [81]:
x = final_df_with_age['Avg age']
y = final_df_with_age['Trauma Percentage']
r, p_value = pearson_correlation_with_pvalue_debug(x, y)
print("Pearson correlation coefficient:", r)
print("P-value:", p_value)


mean_x: 30.24137931034483, mean_y: 64.15111893276588
covariance: -19.660883759464514
std_x: 11.800058445208863, std_y: 120.49389718214158
Pearson correlation coefficient: -0.01382782345201866
P-value: 0.00019121784248588014


There is a very small relation between age and trauma rates. It is not statistically significant enough to matter. However this seems to show that injury rates go down as age goes up? There could be a miscalculation in my pearson coefficient.

In [82]:
x = final_df_with_age['Avg age']
y = final_df_with_age['Deprivation']
r, p_value = pearson_correlation_with_pvalue_debug(x, y)
print("Pearson correlation coefficient:", r)
print("P-value:", p_value)


mean_x: 30.24137931034483, mean_y: 5.0613530720860105
covariance: 90.9512168757183
std_x: 11.800058445208863, std_y: 13.681654884505182
Pearson correlation coefficient: 0.5633596324930059
P-value: 0.3475764169228941


The P value is ridiculous but there is likely to be a correlation between deprivation and age.

In [83]:
df_smoking_status = dftrusts[dftrusts['Dimension'] == 'SmokingStatusGroupBooking']
df_smoking_status = df_smoking_status[['Org_Name', 'Dimension', 'Measure', 'Final_value']]
df_smoking_status.head()

,Org_Name,Dimension,Measure,Final_value
33547,MANCHESTER UNIVERSITY NHS FOUNDATION TRUST,SmokingStatusGroupBooking,Non-Smoker / Ex-Smoker,1095
33548,MANCHESTER UNIVERSITY NHS FOUNDATION TRUST,SmokingStatusGroupBooking,Smoker,95
33654,SOUTH TYNESIDE AND SUNDERLAND NHS FOUNDATION T...,SmokingStatusGroupBooking,Non-Smoker / Ex-Smoker,385
33655,SOUTH TYNESIDE AND SUNDERLAND NHS FOUNDATION T...,SmokingStatusGroupBooking,Smoker,65
33763,UNIVERSITY HOSPITALS DORSET NHS FOUNDATION TRUST,SmokingStatusGroupBooking,Non-Smoker / Ex-Smoker,330


In [84]:
df_smokers = df_smoking_status[df_smoking_status['Measure'] == 'Smoker']
total_final_values = df_smoking_status.groupby('Org_Name')['Final_value'].sum().reset_index()
df_smokers = df_smokers.merge(total_final_values, on='Org_Name', suffixes=('', '_Total'))
df_smokers['Smoker_Percentage'] = (df_smokers['Final_value'] / df_smokers['Final_value_Total']) * 100
df_smokers = df_smokers[['Org_Name', 'Smoker_Percentage']]
df_smokers.head(200)

,Org_Name,Smoker_Percentage
0,MANCHESTER UNIVERSITY NHS FOUNDATION TRUST,7.983193
1,SOUTH TYNESIDE AND SUNDERLAND NHS FOUNDATION T...,14.444444
2,UNIVERSITY HOSPITALS DORSET NHS FOUNDATION TRUST,9.589041
3,ISLE OF WIGHT NHS TRUST,18.750000
4,BARTS HEALTH NHS TRUST,3.930131
...,...,...
116,IMPERIAL COLLEGE HEALTHCARE NHS TRUST,4.081633
117,UNIVERSITY HOSPITALS SUSSEX NHS FOUNDATION TRUST,6.569343
118,AIREDALE NHS TRUST,11.764706
119,NOTTINGHAM UNIVERSITY HOSPITALS NHS TRUST - CI...,14.285714


Let's check for smoking and deprivation, not sure what's going on with chelsea and westminster if you look at the above. 88% smokers in the maternity unit? Nuts. I've just realised I should've made a percentage function, I don't want to rewrite my code so I will write the function below and use it from here on but I will not rewrite previous percentage calculations.

In [85]:
final_df_with_smokers = final_df_with_age.merge(df_smokers, on='Org_Name', how='left')
final_df_with_smokers.head(200)

,Org_Name,Deprivation,Trauma Percentage,Inverted Deprivation,Avg age,Smoker_Percentage
0,ASHFORD AND ST PETER'S HOSPITALS NHS FOUNDATIO...,7.218182,69.565217,0.830999,31,9.433962
1,"BARKING, HAVERING AND REDBRIDGE UNIVERSITY HOS...",4.292308,54.838710,3.756873,30,5.384615
2,BARNSLEY HOSPITAL NHS FOUNDATION TRUST,3.723404,62.962963,4.325776,29,14.893617
3,BARTS HEALTH NHS TRUST,3.22619,36.434109,4.82299,30,3.930131
4,BEDFORDSHIRE HOSPITALS NHS FOUNDATION TRUST,5.4,77.586207,2.64918,31,7.368421
...,...,...,...,...,...,...
111,WHITTINGTON HEALTH NHS TRUST,4.114754,55.555556,3.934426,32,6.451613
112,WIRRAL UNIVERSITY TEACHING HOSPITAL NHS FOUNDA...,4.166667,55.555556,3.882514,29,10.000000
113,WORCESTERSHIRE ACUTE HOSPITALS NHS TRUST,5.744444,68.888889,2.304736,30,13.483146
114,"WRIGHTINGTON, WIGAN AND LEIGH NHS FOUNDATION T...",4.514286,54.545455,3.534895,29,8.571429


In [86]:
x = final_df_with_smokers['Avg age'].tolist()
y = final_df_with_smokers['Smoker_Percentage'].tolist()
r, p_value = pearson_correlation_with_pvalue_debug(x, y)
print("Pearson correlation coefficient:", r)
print("P-value:", p_value)

AttributeError: 'list' object has no attribute 'tolist'

As we have nan values we will drop these and print which trusts have been dropped

In [87]:
nan_orgs = final_df_with_smokers[final_df_with_smokers['Smoker_Percentage'].isna()]['Org_Name']
print("Organizations with NaN in Smoker_Percentage that will be dropped:")
print(nan_orgs.tolist())
final_df_with_smokers= final_df_with_smokers.dropna(subset=['Smoker_Percentage'])

Organizations with NaN in Smoker_Percentage that will be dropped:
['EAST KENT HOSPITALS UNIVERSITY NHS FOUNDATION TRUST']


East Kent of course, they're permanently misreporting. I'll upload another analysis shortly 'Discharge analysis' where we find a few issues with east kent + Medway

In [88]:
x = final_df_with_smokers['Avg age'].tolist()
y = final_df_with_smokers['Smoker_Percentage'].tolist()
r, p_value = pearson_correlation_with_pvalue_debug(x, y)
print("Pearson correlation coefficient:", r)
print("P-value:", p_value)

AttributeError: 'list' object has no attribute 'tolist'

As age increases, rates of smoking decrease slightly.

In [89]:
x = final_df_with_smokers['Inverted Deprivation'].tolist()
y = final_df_with_smokers['Smoker_Percentage'].tolist()
r, p_value = pearson_correlation_with_pvalue_debug(x, y)
print("Pearson correlation coefficient:", r)
print("P-value:", p_value)

AttributeError: 'list' object has no attribute 'tolist'

As deprivation increases, so does smoking rates. The correlation is slight but it is of statistical significance. We're hoping for a significant finding at some point here.

We will look at complex social factors compared to smoking and folic acid supplementation, as well as their relation to age and deprivation. We hypothesise an association here. If we do not find one then we're running out of data to run.

We will remove missing values, if a trust has more than 5% missing values for complex social factors they will be removed from the calculation but we will list removed trusts.

In [90]:
df_csf = dftrusts[dftrusts['Dimension'] == 'ComplexSocialFactorsInd']
df_csf_grouped = df_csf.groupby(['Org_Name', 'Measure'])['Final_value'].sum().unstack()
df_csf_grouped['Total'] = df_csf_grouped['N'] + df_csf_grouped['Y'] + df_csf_grouped['Missing Value']
df_csf_grouped['Missing_Percentage'] = df_csf_grouped.apply(lambda row: calculate_percentage(row['Missing Value'], row['Total']), axis=1)
df_csf_grouped['csfpercent'] = df_csf_grouped.apply(lambda row: calculate_percentage(row['Y'], row['N']), axis=1)
df_csf_grouped.loc['Missing_Percentage'] = df_csf_grouped['Missing_Percentage']
df_csf_grouped.reset_index(inplace=True)
df_csf_grouped = df_csf_grouped[['Org_Name', 'csfpercent', 'Missing_Percentage']]
df_csf_grouped.head(110)

Measure,Org_Name,csfpercent,Missing_Percentage
0,AIREDALE NHS FOUNDATION TRUST,5.714286,NaN
1,AIREDALE NHS TRUST,3.846154,NaN
2,ASHFORD AND ST PETER'S HOSPITALS NHS FOUNDATIO...,14.583333,NaN
3,"BARKING, HAVERING AND REDBRIDGE UNIVERSITY HOS...",20.370370,NaN
4,BARNSLEY HOSPITAL NHS FOUNDATION TRUST,11.627907,NaN
...,...,...,...
105,UNIVERSITY HOSPITALS BRISTOL AND WESTON NHS FO...,9.677419,NaN
106,UNIVERSITY HOSPITALS COVENTRY AND WARWICKSHIRE...,32.558140,0.869565
107,UNIVERSITY HOSPITALS DORSET NHS FOUNDATION TRUST,10.447761,NaN
108,UNIVERSITY HOSPITALS OF DERBY AND BURTON NHS F...,1.685393,NaN


We're going to get a few more variables ready to process (Ethnicity, delivery method) and from there we can do a multivariate analysis to find some strong associations.

In [91]:
final_df_csfpercent = final_df_with_smokers.merge(df_csf_grouped, on='Org_Name', how='left')

# Display the merged DataFrame to verify
final_df_csfpercent.head()


,Org_Name,Deprivation,Trauma Percentage,Inverted Deprivation,Avg age,Smoker_Percentage,csfpercent,Missing_Percentage
0,ASHFORD AND ST PETER'S HOSPITALS NHS FOUNDATIO...,7.218182,69.565217,0.830999,31,9.433962,14.583333,NaN
1,"BARKING, HAVERING AND REDBRIDGE UNIVERSITY HOS...",4.292308,54.838710,3.756873,30,5.384615,20.370370,NaN
2,BARNSLEY HOSPITAL NHS FOUNDATION TRUST,3.723404,62.962963,4.325776,29,14.893617,11.627907,NaN
3,BARTS HEALTH NHS TRUST,3.22619,36.434109,4.82299,30,3.930131,49.704142,0.393701
4,BEDFORDSHIRE HOSPITALS NHS FOUNDATION TRUST,5.4,77.586207,2.64918,31,7.368421,46.969697,NaN


In [92]:
df_ethnic_category = dftrusts[dftrusts['Dimension'] == 'EthnicCategoryMotherGroup']

df_ethnic_category = df_ethnic_category[['Dimension', 'Org_Name', 'Measure', 'Final_value']]
for column in df_ethnic_category.columns:
    if df_ethnic_category[column].dtype == 'bool':
        df_ethnic_category[column] = df_ethnic_category[column].astype(int)

df_ethnic_category.head(100)


,Dimension,Org_Name,Measure,Final_value
33497,EthnicCategoryMotherGroup,MANCHESTER UNIVERSITY NHS FOUNDATION TRUST,Any other ethnic group,65
33498,EthnicCategoryMotherGroup,MANCHESTER UNIVERSITY NHS FOUNDATION TRUST,Asian or Asian British,290
33499,EthnicCategoryMotherGroup,MANCHESTER UNIVERSITY NHS FOUNDATION TRUST,Black or Black British,165
33501,EthnicCategoryMotherGroup,MANCHESTER UNIVERSITY NHS FOUNDATION TRUST,Mixed,60
33502,EthnicCategoryMotherGroup,MANCHESTER UNIVERSITY NHS FOUNDATION TRUST,Not Stated,45
...,...,...,...,...
35270,EthnicCategoryMotherGroup,WIRRAL UNIVERSITY TEACHING HOSPITAL NHS FOUNDA...,Black or Black British,5
35271,EthnicCategoryMotherGroup,WIRRAL UNIVERSITY TEACHING HOSPITAL NHS FOUNDA...,Mixed,5
35272,EthnicCategoryMotherGroup,WIRRAL UNIVERSITY TEACHING HOSPITAL NHS FOUNDA...,Not Stated,5
35273,EthnicCategoryMotherGroup,WIRRAL UNIVERSITY TEACHING HOSPITAL NHS FOUNDA...,White,225


Doing one-hot encoding so that we can add these variables to a multivariable analysis as they are non-numeric,  variables, we will do the same for method of birth. Once we've done this we'll chuck all our data into 

The below methods of transformation are very roundabout, this is because I initially was going to turn all the values into boolean, and then decided to opt for the below instead.

Again, as this is a technical project I feel commited to showing technical capability, so doing things in unnecessarily advanced ways in order to achieve simple objectives is, if anything, positive.

In [93]:
df_encoded = pd.get_dummies(df_ethnic_category, columns=['Measure'])
for column in df_encoded.columns:
    if df_encoded[column].dtype == 'bool':
        df_encoded[column] = df_encoded[column].astype(int)
df_encoded = df_encoded.drop(columns=['Measure_Not known', 'Measure_Not Stated'], errors='ignore')
for index, row in df_encoded.iterrows():
    for col in df_encoded.columns:
        if row[col] == 1:
            df_encoded.at[index, col] = row['Final_value']
df_encoded = df_encoded.groupby('Org_Name').max()
df_encoded.head(50)

,Dimension,Final_value,Measure_Any other ethnic group,Measure_Asian or Asian British,Measure_Black or Black British,Measure_Mixed,Measure_White
Org_Name,,,,,,,
AIREDALE NHS FOUNDATION TRUST,EthnicCategoryMotherGroup,135,0,45,5,5,135
AIREDALE NHS TRUST,EthnicCategoryMotherGroup,95,0,35,5,5,95
ASHFORD AND ST PETER'S HOSPITALS NHS FOUNDATION TRUST,EthnicCategoryMotherGroup,185,10,50,5,5,185
"BARKING, HAVERING AND REDBRIDGE UNIVERSITY HOSPITALS NHS TRUST",EthnicCategoryMotherGroup,280,20,235,65,25,280
BARNSLEY HOSPITAL NHS FOUNDATION TRUST,EthnicCategoryMotherGroup,210,5,5,5,5,210
BARTS HEALTH NHS TRUST,EthnicCategoryMotherGroup,645,65,645,140,25,320
BEDFORDSHIRE HOSPITALS NHS FOUNDATION TRUST,EthnicCategoryMotherGroup,275,15,130,40,10,275
BIRMINGHAM WOMEN'S AND CHILDREN'S NHS FOUNDATION TRUST,EthnicCategoryMotherGroup,280,40,255,75,30,280
BLACKPOOL TEACHING HOSPITALS NHS FOUNDATION TRUST,EthnicCategoryMotherGroup,215,10,10,5,5,215


In [94]:
df_delivery_method = dftrusts[dftrusts['Dimension'] == 'DeliveryMethodBabyGroup']
df_delivery_method = df_delivery_method[['Dimension', 'Org_Name', 'Measure', 'Final_value']]
df_delivery_method_encoded = pd.get_dummies(df_delivery_method, columns=['Measure'])
for column in df_delivery_method_encoded.columns:
    if df_delivery_method_encoded[column].dtype == 'bool':
        df_delivery_method_encoded[column] = df_delivery_method_encoded[column].astype(int)
df_delivery_method_encoded = df_delivery_method_encoded.drop(columns=['Measure_Other'], errors='ignore')
for index, row in df_delivery_method_encoded.iterrows():
    final_value = row['Final_value']  # Capture the Final_value for this row
    for col in df_delivery_method_encoded.columns:
        if row[col] == 1:
            df_delivery_method_encoded.at[index, col] = final_value
df_delivery_method_encoded = df_delivery_method_encoded.drop(columns='Final_value')
df_delivery_method_encoded = df_delivery_method_encoded.groupby('Org_Name').max()
df_delivery_method_encoded1 = df_delivery_method_encoded
# Check for duplicate 'Org_Name' in the index before resetting
if 'Org_Name' in df_delivery_method_encoded.index.names:
    duplicate_check = df_delivery_method_encoded.index.get_level_values('Org_Name').duplicated().any()
    print("Duplicates in Org_Name index:", duplicate_check)

df_delivery_method_encoded1
#we will translate these into percentages

Duplicates in Org_Name index: False


,Dimension,Measure_Elective caesarean section,Measure_Emergency caesarean section,Measure_Instrumental,Measure_Spontaneous
Org_Name,,,,,
AIREDALE NHS FOUNDATION TRUST,DeliveryMethodBabyGroup,30,25,15,80
AIREDALE NHS TRUST,DeliveryMethodBabyGroup,30,25,15,75
ASHFORD AND ST PETER'S HOSPITALS NHS FOUNDATION TRUST,DeliveryMethodBabyGroup,65,65,20,90
"BARKING, HAVERING AND REDBRIDGE UNIVERSITY HOSPITALS NHS TRUST",DeliveryMethodBabyGroup,85,140,40,280
BARNSLEY HOSPITAL NHS FOUNDATION TRUST,DeliveryMethodBabyGroup,5,35,10,130
...,...,...,...,...,...
WIRRAL UNIVERSITY TEACHING HOSPITAL NHS FOUNDATION TRUST,DeliveryMethodBabyGroup,45,50,25,110
WORCESTERSHIRE ACUTE HOSPITALS NHS TRUST,DeliveryMethodBabyGroup,65,60,35,190
"WRIGHTINGTON, WIGAN AND LEIGH NHS FOUNDATION TRUST",DeliveryMethodBabyGroup,35,35,5,80


In [95]:
if 'Org_Name' in df_delivery_method_encoded.index.names:
    df_delivery_method_encoded.fillna(value=0, inplace=True)
df_delivery_method_encoded[2:5] = df_delivery_method_encoded[2:5].apply(pd.to_numeric, errors='coerce')
numeric_cols = df_delivery_method_encoded.select_dtypes(include=[np.number]).columns.tolist()
df_delivery_method_encoded['Row_Sum'] = df_delivery_method_encoded[numeric_cols].sum(axis=1)

# Convert numeric columns to percentages of their row sums
for col in numeric_cols:
    df_delivery_method_encoded[col] = df_delivery_method_encoded[col] / df_delivery_method_encoded['Row_Sum'] * 100

# Drop the 'Row_Sum' column as it's no longer needed after conversion
df_delivery_method_encoded.drop('Row_Sum', axis=1, inplace=True)
df_delivery_method_encoded

,Dimension,Measure_Elective caesarean section,Measure_Emergency caesarean section,Measure_Instrumental,Measure_Spontaneous
Org_Name,,,,,
AIREDALE NHS FOUNDATION TRUST,DeliveryMethodBabyGroup,20.000000,16.666667,10.000000,53.333333
AIREDALE NHS TRUST,DeliveryMethodBabyGroup,20.689655,17.241379,10.344828,51.724138
ASHFORD AND ST PETER'S HOSPITALS NHS FOUNDATION TRUST,NaN,27.083333,27.083333,8.333333,37.500000
"BARKING, HAVERING AND REDBRIDGE UNIVERSITY HOSPITALS NHS TRUST",NaN,15.596330,25.688073,7.339450,51.376147
BARNSLEY HOSPITAL NHS FOUNDATION TRUST,NaN,2.777778,19.444444,5.555556,72.222222
...,...,...,...,...,...
WIRRAL UNIVERSITY TEACHING HOSPITAL NHS FOUNDATION TRUST,DeliveryMethodBabyGroup,19.565217,21.739130,10.869565,47.826087
WORCESTERSHIRE ACUTE HOSPITALS NHS TRUST,DeliveryMethodBabyGroup,18.571429,17.142857,10.000000,54.285714
"WRIGHTINGTON, WIGAN AND LEIGH NHS FOUNDATION TRUST",DeliveryMethodBabyGroup,22.580645,22.580645,3.225806,51.612903


In [96]:
if 'Org_Name' in df_delivery_method_encoded.index.names:
    df_delivery_method_encoded.fillna(value=0, inplace=True)

# Check if 'Org_Name' is in the index and reset it if necessary for df_delivery_method_encoded1
if 'Org_Name' in df_delivery_method_encoded1.index.names:
    df_delivery_method_encoded.fillna(value=0, inplace=True)

print("Columns in df_delivery_method_encoded:", df_delivery_method_encoded.columns)
print("Columns in df_delivery_method_encoded1:", df_delivery_method_encoded1.columns)

org_names_in_encoded = set(df_delivery_method_encoded['Org_Name'])
org_names_in_encoded1 = set(df_delivery_method_encoded1['Org_Name'])

# Find Org_Names that are in df_delivery_method_encoded1 but not in df_delivery_method_encoded
missing_org_names = org_names_in_encoded1.difference(org_names_in_encoded)

# Print the missing Org_Names
print("Org_Names missing from df_delivery_method_encoded:")
print(missing_org_names)


Columns in df_delivery_method_encoded: Index(['Dimension', 'Measure_Elective caesarean section',
       'Measure_Emergency caesarean section', 'Measure_Instrumental',
       'Measure_Spontaneous'],
      dtype='object')
Columns in df_delivery_method_encoded1: Index(['Dimension', 'Measure_Elective caesarean section',
       'Measure_Emergency caesarean section', 'Measure_Instrumental',
       'Measure_Spontaneous'],
      dtype='object')


KeyError: 'Org_Name'

In [92]:
df_delivery_method_encoded

,Org_Name,Dimension,Measure_Elective caesarean section,Measure_Emergency caesarean section,Measure_Instrumental,Measure_Spontaneous
0,AIREDALE NHS FOUNDATION TRUST,DeliveryMethodBabyGroup,20.000000,16.666667,10.000000,53.333333
1,AIREDALE NHS TRUST,DeliveryMethodBabyGroup,20.689655,17.241379,10.344828,51.724138
2,ASHFORD AND ST PETER'S HOSPITALS NHS FOUNDATIO...,NaN,27.083333,27.083333,8.333333,37.500000
3,"BARKING, HAVERING AND REDBRIDGE UNIVERSITY HOS...",NaN,15.596330,25.688073,7.339450,51.376147
4,BARNSLEY HOSPITAL NHS FOUNDATION TRUST,NaN,2.777778,19.444444,5.555556,72.222222
...,...,...,...,...,...,...
118,WIRRAL UNIVERSITY TEACHING HOSPITAL NHS FOUNDA...,DeliveryMethodBabyGroup,19.565217,21.739130,10.869565,47.826087
119,WORCESTERSHIRE ACUTE HOSPITALS NHS TRUST,DeliveryMethodBabyGroup,18.571429,17.142857,10.000000,54.285714
120,"WRIGHTINGTON, WIGAN AND LEIGH NHS FOUNDATION T...",DeliveryMethodBabyGroup,22.580645,22.580645,3.225806,51.612903
121,WYE VALLEY NHS TRUST,DeliveryMethodBabyGroup,22.727273,27.272727,9.090909,40.909091


In [93]:
df_delivery_method_encoded1

,Org_Name,Dimension,Measure_Elective caesarean section,Measure_Emergency caesarean section,Measure_Instrumental,Measure_Spontaneous
0,AIREDALE NHS FOUNDATION TRUST,DeliveryMethodBabyGroup,20.000000,16.666667,10.000000,53.333333
1,AIREDALE NHS TRUST,DeliveryMethodBabyGroup,20.689655,17.241379,10.344828,51.724138
2,ASHFORD AND ST PETER'S HOSPITALS NHS FOUNDATIO...,NaN,27.083333,27.083333,8.333333,37.500000
3,"BARKING, HAVERING AND REDBRIDGE UNIVERSITY HOS...",NaN,15.596330,25.688073,7.339450,51.376147
4,BARNSLEY HOSPITAL NHS FOUNDATION TRUST,NaN,2.777778,19.444444,5.555556,72.222222
...,...,...,...,...,...,...
118,WIRRAL UNIVERSITY TEACHING HOSPITAL NHS FOUNDA...,DeliveryMethodBabyGroup,19.565217,21.739130,10.869565,47.826087
119,WORCESTERSHIRE ACUTE HOSPITALS NHS TRUST,DeliveryMethodBabyGroup,18.571429,17.142857,10.000000,54.285714
120,"WRIGHTINGTON, WIGAN AND LEIGH NHS FOUNDATION T...",DeliveryMethodBabyGroup,22.580645,22.580645,3.225806,51.612903
121,WYE VALLEY NHS TRUST,DeliveryMethodBabyGroup,22.727273,27.272727,9.090909,40.909091


In [99]:
for column in df_encoded.columns:
    if df_encoded[column].dtype == 'bool':
        df_encoded[column] = df_encoded[column].astype(int)
numeric_cols = df_encoded.select_dtypes(include=[np.number]).columns.tolist()
df_encoded['Row_Sum'] = df_encoded[numeric_cols].sum(axis=1)
for col in numeric_cols:
    df_encoded[col] = df_encoded[col] / df_encoded['Row_Sum'] * 100
df_encoded.drop('Row_Sum', axis=1, inplace=True)
df_encoded = df_encoded.groupby('Org_Name').max()
if 'Org_Name' in df_encoded.index.names:
    df_encoded.reset_index(inplace=True)
if 'Final_value' in df_encoded.columns:
    df_encoded = df_encoded.drop(columns='Final_value')
df_encoded.head(100)

,Org_Name,Dimension,Measure_Any other ethnic group,Measure_Asian or Asian British,Measure_Black or Black British,Measure_Mixed,Measure_White
0,AIREDALE NHS FOUNDATION TRUST,EthnicCategoryMotherGroup,0.000000,23.684211,2.631579,2.631579,71.052632
1,AIREDALE NHS TRUST,EthnicCategoryMotherGroup,0.000000,25.000000,3.571429,3.571429,67.857143
2,ASHFORD AND ST PETER'S HOSPITALS NHS FOUNDATIO...,EthnicCategoryMotherGroup,3.921569,19.607843,1.960784,1.960784,72.549020
3,"BARKING, HAVERING AND REDBRIDGE UNIVERSITY HOS...",EthnicCategoryMotherGroup,3.200000,37.600000,10.400000,4.000000,44.800000
4,BARNSLEY HOSPITAL NHS FOUNDATION TRUST,EthnicCategoryMotherGroup,2.173913,2.173913,2.173913,2.173913,91.304348
...,...,...,...,...,...,...,...
95,THE PRINCESS ALEXANDRA HOSPITAL NHS TRUST,EthnicCategoryMotherGroup,1.538462,4.615385,6.153846,1.538462,86.153846
96,"THE QUEEN ELIZABETH HOSPITAL, KING'S LYNN, NHS...",EthnicCategoryMotherGroup,3.225806,3.225806,3.225806,3.225806,87.096774
97,THE ROTHERHAM NHS FOUNDATION TRUST,EthnicCategoryMotherGroup,5.714286,11.428571,2.857143,2.857143,77.142857
98,THE ROYAL WOLVERHAMPTON NHS TRUST,EthnicCategoryMotherGroup,3.960396,18.811881,11.881188,3.960396,61.386139


In [99]:
df_delivery_method_encoded = df_delivery_method_encoded.drop(columns=['Dimension'], errors='ignore')
df_encoded = df_encoded.drop(columns=['Dimension'], errors='ignore')

In [119]:
merged_df = pd.merge(df_encoded, df_delivery_method_encoded, on='Org_Name', how='outer')
centraldf = pd.merge(merged_df, final_df_csfpercent, on='Org_Name', how='outer')
#I'm going to drop Airedale as they have so many nan values
centraldf = centraldf[~centraldf['Org_Name'].str.startswith('AIREDALE')]
#Fixed
centraldf = centraldf.drop(columns=['Missing_Percentage'], errors='ignore')
centraldf = centraldf.drop(columns=['Final_value'], errors='ignore')
centraldf

,Org_Name,Measure_Any other ethnic group,Measure_Asian or Asian British,Measure_Black or Black British,Measure_Mixed,Measure_White,Measure_Elective caesarean section,Measure_Emergency caesarean section,Measure_Instrumental,Measure_Spontaneous,Deprivation,Trauma Percentage,Inverted Deprivation,Avg age,Smoker_Percentage,csfpercent
2,ASHFORD AND ST PETER'S HOSPITALS NHS FOUNDATIO...,10,50,5,5,185,27.08,27.08,8.33,37.50,7.22,69.57,0.83,31.00,9.43,14.58
3,"BARKING, HAVERING AND REDBRIDGE UNIVERSITY HOS...",20,235,65,25,280,15.60,25.69,7.34,51.38,4.29,54.84,3.76,30.00,5.38,20.37
4,BARNSLEY HOSPITAL NHS FOUNDATION TRUST,5,5,5,5,210,2.78,19.44,5.56,72.22,3.72,62.96,4.33,29.00,14.89,11.63
5,BARTS HEALTH NHS TRUST,65,645,140,25,320,11.73,21.94,11.22,55.10,3.23,36.43,4.82,30.00,3.93,49.70
6,BEDFORDSHIRE HOSPITALS NHS FOUNDATION TRUST,15,130,40,10,275,18.18,22.22,8.08,51.52,5.40,77.59,2.65,31.00,7.37,46.97
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120,WIRRAL UNIVERSITY TEACHING HOSPITAL NHS FOUNDA...,5,5,5,5,225,19.57,21.74,10.87,47.83,4.17,55.56,3.88,29.00,10.00,6.38
121,WORCESTERSHIRE ACUTE HOSPITALS NHS TRUST,5,30,5,5,365,18.57,17.14,10.00,54.29,5.74,68.89,2.30,30.00,13.48,9.88
122,"WRIGHTINGTON, WIGAN AND LEIGH NHS FOUNDATION T...",5,5,10,5,155,22.58,22.58,3.23,51.61,4.51,54.55,3.53,29.00,8.57,12.90
123,WYE VALLEY NHS TRUST,0,5,5,5,125,22.73,27.27,9.09,40.91,5.35,75.00,2.69,30.00,10.34,11.54


We will now find correlations between all of our data

In [ ]:
correlation = centraldf.copy()
correlation = correlation.drop(columns=['Org_Name'])

# Define columns of interest
columns_of_interest = [
    'Measure_Elective caesarean section',
    'Measure_Emergency caesarean section',
    'Inverted Deprivation',
    'Trauma Percentage',
    'Smoker_Percentage',
    'csfpercent'
]

# Dictionary to store results
results = []

for col1 in columns_of_interest:
    for col2 in correlation.columns:
        if col1 != col2 and col2 not in columns_of_interest:
            x = correlation[col1].dropna()
            y = correlation[col2].dropna()
            
            # Align x and y by their index
            common_index = x.index.intersection(y.index)
            x = x.loc[common_index]
            y = y.loc[common_index]
            
            # Ensure both series have the same length and are non-empty
            if len(x) > 0 and len(y) > 0 and len(x) == len(y):
                r, p_value = pearson_correlation_with_pvalue_debug(x, y)
                results.append({
                    'Measure': col1,
                    'ComparedMeasure': col2,
                    'Pearson Correlation': r,
                    'P-value': p_value
                })

# Convert results to a DataFrame
correlationfin = pd.DataFrame(results)
pd.options.display.float_format = '{:.2f}'.format

correlationfin



In [129]:
import statsmodels.api as sm
centraldf_filled = centraldf
centraldf_filled = centraldf_filled.drop(columns=['Org_Name'], errors='ignore')

centraldf_filled = centraldf_filled.fillna(centraldf_filled.mean())

# Define the dependent variable
Y = centraldf_filled['Trauma Percentage']

# Define the independent variables
X = centraldf_filled[['Smoker_Percentage', 'csfpercent', 'Measure_Elective caesarean section', 
                      'Measure_Emergency caesarean section', 'Inverted Deprivation', 'Avg age']]

# Add a constant (intercept) to the model
X = sm.add_constant(X)

# Fit the regression model
model = sm.OLS(Y, X).fit()

# Print the model summary
print(model.summary())


                            OLS Regression Results                            
Dep. Variable:      Trauma Percentage   R-squared:                       0.108
Model:                            OLS   Adj. R-squared:                  0.061
Method:                 Least Squares   F-statistic:                     2.330
Date:                Tue, 27 Aug 2024   Prob (F-statistic):             0.0368
Time:                        17:23:53   Log-Likelihood:                -460.21
No. Observations:                 123   AIC:                             934.4
Df Residuals:                     116   BIC:                             954.1
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                                          coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------
co